# Machine Learning HW.3 

# CNN-Classification

Dataset: https://www.kaggle.com/c/ml2020spring-hw3/data

In [1]:
import os
import cv2
import numpy as np
import pandas as pd
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optimz
import torchvision.transforms as transforms
from torch.utils.data import DataLoader,Dataset

In [2]:
def read_photo(path,label = False):
    path_list = os.listdir(path)
    x = np.zeros((len(path_list), 128, 128, 3), dtype = np.uint8)
    y = np.zeros((len(path_list)), dtype = np.uint8)
    for i,f in enumerate(path_list):
        img = cv2.imread(os.path.join(path, f))
        x[i,:,:] = cv2.resize(img, (128, 128))
        if label:
            y[i] = int(f.split('_')[0])
    if label:
        return x, y
    else:
        return x

In [3]:
path = '/Users/Juyi/Desktop/Code/Data/food-11'
train_data, train_label = read_photo(os.path.join(path, 'training'), True)
val_data, val_label = read_photo(os.path.join(path, 'validation'), True)
test_data = read_photo(os.path.join(path, 'testing'))

In [4]:
print(train_data.shape,train_label.shape)
print(val_data.shape,val_label.shape)
print(test_data.shape)

(9866, 128, 128, 3) (9866, 1)
(3430, 128, 128, 3) (3430, 1)
(3110, 128, 128, 3)


In [4]:
train_transforms = transforms.Compose([transforms.ToPILImage(), transforms.RandomHorizontalFlip(), 
        transforms.RandomRotation(15), transforms.ToTensor(),])
test_transforms = transforms.Compose([transforms.ToPILImage(), transforms.ToTensor(),])

class ImgData(Dataset):
    def __init__(self, x, y = None, transform = None):
        self.x = x
        self.y = y
        if y is not None:
            self.y = torch.LongTensor(y)
        self.transforms = transform

    def __getitem__(self, index):
        X = self.x[index]
        if self.transforms is not None:
            X = self.transforms(X)
        if self.y is not None:
            Y = self.y[index]
            return X,Y
        else:
            return X

    def __len__(self):
        return len(self.x)

In [5]:
batch_size = 128
train_set = ImgData(train_data, train_label, train_transforms)
val_set = ImgData(val_data, val_label, test_transforms)
test_set = ImgData(test_data,transform = test_transforms)

train_loader = DataLoader(train_set, batch_size = batch_size, shuffle = True)
val_loader = DataLoader(val_set, batch_size = batch_size, shuffle = False)
test_loader = DataLoader(test_set, batch_size = 128, shuffle = False)

In [16]:
class Cnn(nn.Module):
    def __init__(self):
        super(Cnn, self).__init__()
        # torch.nn.Conv2d(in_channels, out_channels, kernel_size, stride, padding)
        # torch.nn.MaxPool2d(kernel_size, stride, padding)
        # input 维度 [3, 128, 128]
        self.cnn = nn.Sequential(
            nn.Conv2d(3, 64, 3, 1, 1),      # [64, 128, 128]
            nn.BatchNorm2d(64),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),          # [64, 64, 64]

            nn.Conv2d(64, 128, 3, 1, 1),    # [128, 64, 64]
            nn.BatchNorm2d(128),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),          # [128, 32, 32]

            nn.Conv2d(128, 256, 3, 1, 1),   # [256, 32, 32]
            nn.BatchNorm2d(256),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),          # [256, 16, 16]

            nn.Conv2d(256, 512, 3, 1, 1),# [512, 16, 16]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0),          # [512, 8, 8]

            nn.Conv2d(512, 512, 3, 1, 1),# [512, 8,8]
            nn.BatchNorm2d(512),
            nn.ReLU(),
            nn.MaxPool2d(2, 2, 0)           # [512, 4, 4]
        )
        
        self.func = nn.Sequential(
            nn.Linear(512 * 4 * 4, 1024),   # [1024]
            nn.ReLU(),
            nn.Linear(1024, 512),           # [512]
            nn.ReLU(),
            nn.Linear(512, 11)              # [11]
        )
    
    def forward(self, x):
        cnn_out = self.cnn(x)
        cnn_out = cnn_out.view(cnn_out.size()[0], -1)   #flattn
        func_out = self.func(cnn_out)
        return func_out

In [19]:
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
#cnn = Classifier()
cnn = Cnn()
optimzer = optimz.Adam(cnn.parameters(), lr = 0.001)
critization = nn.CrossEntropyLoss()
epoch_num = 2
for epoch in range(epoch_num):
    train_loss = 0
    val_loss = 0
    train_acc = 0
    val_acc = 0
    cnn.train()
    for data,label in train_loader:
        optimzer.zero_grad()
        train_pre = cnn(data)
        batch_loss = critization(train_pre, label)
        batch_loss.backward()
        optimzer.step()
        train_acc += np.sum(np.argmax(train_pre.cpu().data.numpy(), axis = 1) == label.numpy())
        train_loss += batch_loss.item()
    
    print(f'{epoch} Epoch , the loss is {train_loss/train_set.__len__()}, the accuarcy is {train_acc/train_set.__len__()}')
    

RuntimeError: Trying to backward through the graph a second time, but the buffers have already been freed. Specify retain_graph=True when calling backward the first time.